<a href="https://colab.research.google.com/github/chacha86/python-note/blob/main/article_db_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


# 연결

In [ ]:
# 1단계 : import
import pymysql as my

# 2단계 : connect(mysql 접속)
db = my.connect(host='localhost', port=3306, user='root', passwd='', db='b2', charset='utf8')
# 3단계 : cusor : 커서 생성
cursor = db.cursor(my.cursors.DictCursor)



# 게시물 목록

In [ ]:
# 4단계 : 

board_id = input("게시판을 선택해주세요(1. 질문과 답변, 2. 공지사항, 3. 자유게시판) : ")

sql = 'SELECT * FROM article a INNER JOIN member m ON a.member_id = m.member_id where board_id = {} order by a.article_id'.format(board_id)
cursor.execute(sql)

result = cursor.fetchall()

def add_padding(pstr, width) :
    if str(type(pstr)) == "<class 'int'>" :
        pstr = str(pstr)
    
    cnt = 0
    for c in pstr :
        if 0 <= ord(c) and ord(c) <= 127 :
            cnt += 1
        else :
            cnt += 2
    
    pd_len = width - cnt
    for i in range(pd_len) :
        pstr += ' '
    
    return pstr

def f2(s1, s2, s3, s4, s5) :
    if len(s2) >= 15 :
        s2 = s2[:12] + "..."
    print(add_padding(s1, 5), end=" ")
    print(add_padding(s2, 30), end=" ")
    print(add_padding(s3, 10), end=" ")
    print(add_padding(s4, 24), end=" ")
    print(add_padding(s5, 0), end=" ")
    print()
    print("---------------------------------------------------------------------------------")

def print_articles(articles) :
    f2("번호", "제목", "작성자", "작성일", "조회수")
    for a in articles :
        f2(a["article_id"], a["title"], a["nick_name"], str(a['regDate'])[:10], a['hit'])

print_articles(result)

게시판을 선택해주세요(1. 질문과 답변, 2. 공지사항, 3. 자유게시판) : 1
번호  제목                           작성자     작성일                   조회수 
---------------------------------------------------------------------------------
8     제목8                          의적       2020-07-01               87 
---------------------------------------------------------------------------------
9     제목9                          의적       2022-03-03               56 
---------------------------------------------------------------------------------
10    제목10                         해신       2022-04-01               3 
---------------------------------------------------------------------------------
11    파이썬 좀 쉽게 할 수...        길동이     2022-04-04               0 
---------------------------------------------------------------------------------
12    dfdfer                         길동이     2022-04-04               0 
---------------------------------------------------------------------------------


# 게시물 상세보기

In [ ]:
def print_detail(article, replies) :
    print("================== 게시물 상세보기 ===================")
    print("번호 : {}".format(article["article_id"])) 
    print("제목 : {}".format(article["title"]))
    print("내용 : {}".format(article["body"]))
    print("작성자 : {}".format(article["nick_name"]))
    print("작성일 : {}".format(article["regDate"]))      
    print("------------------------------------------------------")
    print("댓글 {}".format(len(replies)))
    print("------------------------------------------------------")
    for reply in replies :
        print("내용 : {}".format(reply["body"]))
        print("작성자 : {}".format(reply["nick_name"]))
        print("작성일 : {}".format(reply["regDate"]))
        print("--------------------------------------------------------")
          
# ===============================================================================================================
id = input("번호를 입력해주세요 : ")

sql = '''
    SELECT *, m.nick_name 
      FROM article a
     INNER JOIN member m
        ON a.member_id = m.member_id
     WHERE a.article_id = {}
    '''.format(id)

cursor.execute(sql)
a = cursor.fetchone()

sql = '''
    SELECT *, m.nick_name
      FROM articleReply ar
     INNER JOIN member m
        ON ar.member_id = m.member_id
     WHERE article_id = {}
'''.format(id)

cursor.execute(sql)
ar = cursor.fetchall()

print_detail(a, ar)

# ===============================================================================================================


번호를 입력해주세요 : 6
================== 게시물 상세보기 ===================
번호 : 6
제목 : 제목6
내용 : 내용6
작성자 : 해신
작성일 : 2020-03-03 12:30:00
------------------------------------------------------
댓글 3
------------------------------------------------------
내용 : 댓글내용3
작성자 : 길동이
작성일 : 2020-03-03 12:30:00
--------------------------------------------------------
내용 : 댓글내용4
작성자 : 길동이
작성일 : 2020-03-05 15:12:00
--------------------------------------------------------
내용 : 댓글내용5
작성자 : 길동이
작성일 : 2020-03-12 19:08:00
--------------------------------------------------------


# 게시물 등록

In [ ]:
loginId = input("아이디 : ")
loginPw = input("비밀번호 : ")

sql = "SELECT * FROM member WHERE login_id = '{}' and login_pw = '{}'".format(loginId, loginPw)

cursor.execute(sql)
member = cursor.fetchone()

if member == None :
    print("없는 아이디입니다.")

else :
    
    print("안녕하세요 {}님!".format(member["nick_name"]))
    board_id = input("게시판을 선택해주세요 (1. 질문과 답변, 2. 공지사항, 3. 자유게시판): ")
    title = input("제목 : ")
    body = input("내용 : ")

    sql = """
        INSERT INTO article
           SET title = '{}'
             , body = '{}'
             , member_id = {}
             , board_id = {}
             , hit = 0
             , regDate = NOW()
    """.format(title, body, member["member_id"], board_id)

    cursor.execute(sql)
    db.commit()


아이디 : user1
비밀번호 : 1234
안녕하세요 길동이님!
게시판을 선택해주세요 (1. 질문과 답변, 2. 공지사항, 3. 자유게시판): 2
제목 : hahahaha
내용 : mamamama


# 게시물 검색

In [ ]:
board_id = input("게시판을 선택해주세요(1. 질문과 답변, 2. 공지사항, 3. 자유게시판) : ")
keyword = input("검색어 : ")

sql = "SELECT * FROM article a INNER JOIN member m ON a.member_id = m.member_id where board_id = {} and title like '%{}%' order by a.article_id".format(board_id, keyword)
print(sql)
cursor.execute(sql)

result = cursor.fetchall()

print_articles(result)

게시판을 선택해주세요(1. 질문과 답변, 2. 공지사항, 3. 자유게시판) : 1
검색어 : 제
SELECT * FROM article a INNER JOIN member m ON a.member_id = m.member_id where board_id = 1 and title like '%제%' order by a.article_id
번호  제목                           작성자     작성일                   조회수 
---------------------------------------------------------------------------------
8     제목8                          의적       2020-07-01               87 
---------------------------------------------------------------------------------
9     제목9                          의적       2022-03-03               56 
---------------------------------------------------------------------------------
10    제목10                         해신       2022-04-01               3 
---------------------------------------------------------------------------------


# 게시물 수정

# 게시물 삭제

# 게시물 정렬